In [ ]:
import sys
import IPython

%pip install --upgrade --user --quiet google-genai
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [ ]:
from google.genai.types import GenerateContentConfig, Part, SafetySetting

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
MODEL_ID =  "meta/llama-4-maverick-17b-128e-instruct-maas"


Defining the format we want to return...

In [ ]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "rating": {"type": "INTEGER"},
                "flavor": {"type": "STRING"},
                "sentiment": {
                    "type": "STRING",
                    "enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"],
                },
                "explanation": {"type": "STRING"},
            },
            "required": ["rating", "flavor", "sentiment", "explanation"],
        },
    },
}


defining the product reviews in the prompt and asking the model to generate based on the response_schema passed in.

In [ ]:
prompt = """
  Analyze the following product reviews, output the sentiment classification, and give an explanation.

  - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
  - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)
product_reviews: dict = response.parsed
print(product_reviews)

[[{'explanation': "The reviewer used the phrase 'Absolutely loved it' and stated it was the 'Best ice cream I've ever had', indicating a very positive sentiment despite the rating being less than 5.", 'flavor': 'Strawberry Cheesecake', 'rating': 4, 'sentiment': 'POSITIVE'}, {'explanation': "The reviewer described the product as 'Quite good', but expressed a negative aspect by stating it was 'a bit too sweet', aligning with the low rating given. The negative aspect outweighs the positive, leading to an overall negative sentiment.", 'flavor': 'Mango Tango', 'rating': 1, 'sentiment': 'NEGATIVE'}]]
